# Problem 1:
### Part 1 – Calculate the maximum SR portfolio using the method from last week’s homework for the following stocks: AAPL, MSFT, BRK-B, CSCO, and JNJ.
### Use the returns from the end of the history (1-14) until the end of February. Calculate the Ex-Post Return Attribution for each Stock.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime
import math
import itertools
from scipy import optimize
import sys
sys.path.append("..")
from RiskPackage.CalculateReturn import return_calculate
from scipy import linalg

## Calculate the optimal Sharpe ratio portfolio
### Using 4-factor Model to calculate the expected return to each stock

In [2]:
# parse the data of fama 3-factor
fama_data=pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
fama_data['Date']=fama_data['Date'].apply(str)
fama_data=fama_data.set_index(['Date'])
fama_data.index=pd.to_datetime(fama_data.index,format='%Y%m%d')
fama_data/=100
# parse the data of momentum
mom_data=pd.read_csv('F-F_Momentum_Factor_daily.CSV')
mom_data['Date']=mom_data['Date'].apply(str)
mom_data=mom_data.set_index(['Date'])
mom_data.index=pd.to_datetime(mom_data.index,format='%Y%m%d')
mom_data/=100
# combine them together
fama_data=pd.concat([fama_data,mom_data],axis=1)

# parse the return data 
rt=pd.read_csv('Dailyreturn.csv',index_col='Date')
rt.index=pd.to_datetime(rt.index)

In [3]:
stock_list=['AAPL','MSFT','BRK-B','CSCO','JNJ']

In [4]:
# find the intersected date between fama data and return data
intersected_date=list(set(rt.index).intersection(set(fama_data.index )))
excess_r=rt.sub(fama_data.loc[intersected_date,'RF'].sort_index(axis=0),axis=0)
# get the return and excess return data of given stock list
rt=rt[stock_list]
excess_r=excess_r[stock_list]

In [5]:
# Ge the X to do the regression (4 facotrs)
x4=pd.concat([fama_data.loc[intersected_date,:'HML'],fama_data.loc[intersected_date,'Mom   ']],axis=1)
x4['intersect']=1
x4=x4.sort_index(axis=0)
# Calculate the beta
X4=np.array(x4)
beta4=linalg.inv(X4.T@X4)@X4.T@excess_r
beta4.index=x4.columns

In [6]:
# past 10 years of factor returns
expected_fama_factor=fama_data.loc[rt.index[-1]-pd.DateOffset(years=10):rt.index[-1],:].mean()
expected_fama_factor['intersect']=1

# past 10 years of risk-free rate
expected_RF=expected_fama_factor['RF']

# 4-factor returns
# expected_fama_factor4 = expected_fama_factor[["Mkt-RF","SMB","HML",'Mom   ',"intersect"]]
beta4=beta4.iloc[:4,:]
expected_fama_factor4 = expected_fama_factor[["Mkt-RF","SMB","HML",'Mom   ']]

# Expected return of each stock
expected_return4=expected_fama_factor4 @ beta4
expected_return4=np.log(1+expected_return4)*255
# Covariance
cov=np.log(1+excess_r).cov()*255

In [7]:
def super_efficient_portfolio(expected_rts,cov):
    '''Given a target return, use assets to find the optimal portfolio with lowest risk'''
    rf=0.0025
    fun=lambda wts: -(wts@expected_rts-rf)/np.sqrt(wts@cov@wts)
    x0 = np.full(expected_rts.shape[0],1/expected_rts.shape[0])
    cons = [{'type':'ineq', 'fun':lambda x:x},
        {'type':'eq', 'fun':lambda x:sum(x)-1}]
    bounds = [(0, 1) for _ in range(expected_rts.shape[0])]
    res = optimize.minimize(fun, x0, method='SLSQP',bounds=bounds,constraints=cons)
    return res

In [8]:
# optimal portfolio
res=super_efficient_portfolio(expected_return4,cov)
SR_Portfolio=pd.DataFrame(index=stock_list)
SR_Portfolio['Weight']=res.x
SR_Portfolio['return_attribution']=res.x*expected_return4

## Realized Return Attribution

In [9]:
# Get Updated Prices 
updated=pd.read_csv('updated_prices.csv',index_col='Date')

#calculate portfolio return and updated weights for each day
rts=return_calculate(updated,rm_means=False)
rts.index=pd.to_datetime(rts.index)

# Use the optimal portforlio weights as initial weights
last_weight=SR_Portfolio['Weight']
weights=[]
portfolio_rts=[]
for i in range(rts.shape[0]):
    # Store the weights
    weights.append(last_weight)

    # Update Weights by return
    last_weight=last_weight*(rts.iloc[i,1:]+1)
    # Calculate the portforlio return
    p_rt=last_weight.sum()
    # Normalize the wieghts back so sum = 1
    last_weight=last_weight/p_rt
    
    # Store the portforlio return
    portfolio_rts.append(p_rt-1)
weights=pd.DataFrame(weights,index=rts.index)
weights.index=pd.to_datetime(weights.index)

# Add the Portfolio Return to the dataframe rts
rts['PortfolioReturn']=pd.DataFrame(portfolio_rts,index=rts.index)
# Calculate the total return
total_rt=(rts['PortfolioReturn']+1).prod()-1

In [10]:
# Calculate the Carino K
k=np.log(total_rt+1)/total_rt
# Carino k_t is the ratio scaled by 1/K 
carinoK = np.log(1+rts['PortfolioReturn'])/k/rts['PortfolioReturn']
# Transform carinoK to dataframe to be multiplied by weights and rts
carinoK_df=pd.DataFrame([carinoK]*weights.shape[1],index=weights.columns).T
# Calculate the return attribution (has been adjusted by carinoK_df)
return_attribution=(weights * rts * carinoK_df).dropna(axis=1)

In [11]:
# Calculate the total return attribution
total_rt_attribution=return_attribution.sum()
# Calculate the total return of each stock and portfolio
total_rts=((rts+1).prod()-1).iloc[1:]
# Combine the total_rt_attribution and total_rts together to compare with each other
attribution_df=pd.concat([total_rt_attribution,total_rts],axis=1).T
attribution_df.index=['TotalReturnAttribution','RealizedTotalReturn']
# Validate whether the sum of Total Return Attribution is equal to the total Portfolio Return or not
# Check that the attribution sums back to the total Portfolio return
attribution_df.loc['TotalReturnAttribution','PortfolioReturn']=attribution_df.loc['TotalReturnAttribution'][:-1].sum()
print(np.isclose(attribution_df.loc['TotalReturnAttribution','PortfolioReturn'],attribution_df.loc['RealizedTotalReturn','PortfolioReturn']))
attribution_df

True


,AAPL,BRK-B,CSCO,JNJ,MSFT,PortfolioReturn
TotalReturnAttribution,-0.00459,-0.003693,-0.007547,-0.002152,-0.007082,-0.025063
RealizedTotalReturn,-0.04472,-0.008268,-0.091102,-0.013189,-0.034791,-0.025063


### Part 2 – Using the same data as part 1, add the risk attribution for each stock:

## Realized Volatility Attribution

In [12]:
# Y is stock returns scaled by their weight at each time
Y = (weights * rts).dropna(axis=1)

# Set up X with the Portfolio Return
X = pd.DataFrame(rts['PortfolioReturn'])
X['Intersect']=1

# Calculate the Beta and discard the intercept
B = (linalg.inv(X.T@X)@X.T@Y).iloc[:-1,:]

# Component SD is Beta times the standard Deviation of the portfolio
cSD = B * rts['PortfolioReturn'].std()

#Check that the sum of component SD is equal to the portfolio SD
print(np.isclose(cSD.sum(axis=1),rts['PortfolioReturn'].std()))

# Add the portfolio SD to the cSD
cSD['Portfolio']=rts['PortfolioReturn'].std()

# Add the Vol attribution to attribution_df
columns=list(attribution_df.columns)
columns[-1]='Portfolio'
attribution_df.columns=columns
attribution_df.loc['VolatilityAttribution',:]=cSD.values
attribution_df

[ True]


,AAPL,BRK-B,CSCO,JNJ,MSFT,Portfolio
TotalReturnAttribution,-0.004590,-0.003693,-0.007547,-0.002152,-0.007082,-0.025063
RealizedTotalReturn,-0.044720,-0.008268,-0.091102,-0.013189,-0.034791,-0.025063
VolatilityAttribution,0.001593,0.004773,0.000910,0.001434,0.003215,0.011926


# Problem 2:
### Using the same data as Problem 1, attribute realized risk and return to the Fama French 3+Momentum model. Report the residual total as Portfolio Alpha.

In [13]:
# Clear the memory
%reset -f

import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime
import math
import itertools
from scipy import optimize
import sys
sys.path.append("..")
from RiskPackage.CalculateReturn import return_calculate
from scipy import linalg

## Calculate the optimal Sharpe ratio portfolio
### Using 4-factor Model to calculate the expected return to each stock

In [14]:
# parse the data of fama 3-factor
fama_data=pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
fama_data['Date']=fama_data['Date'].apply(str)
fama_data=fama_data.set_index(['Date'])
fama_data.index=pd.to_datetime(fama_data.index,format='%Y%m%d')
fama_data/=100
# parse the data of momentum
mom_data=pd.read_csv('F-F_Momentum_Factor_daily.CSV')
mom_data['Date']=mom_data['Date'].apply(str)
mom_data=mom_data.set_index(['Date'])
mom_data.index=pd.to_datetime(mom_data.index,format='%Y%m%d')
mom_data/=100
# combine them together
fama_data=pd.concat([fama_data,mom_data],axis=1)

# parse the return data 
rt=pd.read_csv('Dailyreturn.csv',index_col='Date')
rt.index=pd.to_datetime(rt.index)

In [15]:
stock_list=['AAPL','MSFT','BRK-B','CSCO','JNJ']

In [16]:
# find the intersected date between fama data and return data
intersected_date=list(set(rt.index).intersection(set(fama_data.index )))
excess_r=rt.sub(fama_data.loc[intersected_date,'RF'].sort_index(axis=0),axis=0)
# get the return and excess return data of given stock list
rt=rt[stock_list]
excess_r=excess_r[stock_list]

In [17]:
# Ge the X to do the regression (4 facotrs)
x4=pd.concat([fama_data.loc[intersected_date,:'HML'],fama_data.loc[intersected_date,'Mom   ']],axis=1)
x4['intersect']=1
x4=x4.sort_index(axis=0)
# Calculate the beta
X4=np.array(x4)
beta4=linalg.inv(X4.T@X4)@X4.T@excess_r
beta4.index=x4.columns

In [18]:
# past 10 years of factor returns
expected_fama_factor=fama_data.loc[rt.index[-1]-pd.DateOffset(years=10):rt.index[-1],:].mean()
expected_fama_factor['intersect']=1

# past 10 years of risk-free rate
expected_RF=expected_fama_factor['RF']

# 4-factor returns
# expected_fama_factor4 = expected_fama_factor[["Mkt-RF","SMB","HML",'Mom   ',"intersect"]]
beta4=beta4.iloc[:4,:]
expected_fama_factor4 = expected_fama_factor[["Mkt-RF","SMB","HML",'Mom   ']]

# Expected return of each stock
expected_return4=expected_fama_factor4 @ beta4
expected_return4=np.log(1+expected_return4)*255
# Covariance
cov=np.log(1+excess_r).cov()*255

In [19]:
def super_efficient_portfolio(expected_rts,cov):
    '''Given a target return, use assets to find the optimal portfolio with lowest risk'''
    rf=0.0025
    fun=lambda wts: -(wts@expected_rts-rf)/np.sqrt(wts@cov@wts)
    x0 = np.full(expected_rts.shape[0],1/expected_rts.shape[0])
    cons = [{'type':'ineq', 'fun':lambda x:x},
        {'type':'eq', 'fun':lambda x:sum(x)-1}]
    bounds = [(0, 1) for _ in range(expected_rts.shape[0])]
    res = optimize.minimize(fun, x0, method='SLSQP',bounds=bounds,constraints=cons)
    return res

In [20]:
# optimal portfolio
res=super_efficient_portfolio(expected_return4,cov)
SR_Portfolio=pd.DataFrame(index=stock_list)
SR_Portfolio['Weight']=res.x
SR_Portfolio['return_attribution']=res.x*expected_return4

## Factor Attribution

In [21]:
# Get Updated Prices 
updated=pd.read_csv('updated_prices.csv',index_col='Date')

#calculate portfolio return and updated weights for each day
rts=return_calculate(updated,rm_means=False)
rts.index=pd.to_datetime(rts.index)

# Update the Fama date to the latest
updated_fama3=pd.read_csv('updated_F-F_Research_Data_Factors_daily.CSV')
updated_fama3['Date']=updated_fama3['Date'].apply(str)
updated_fama3=updated_fama3.set_index(['Date'])
updated_fama3.index=pd.to_datetime(updated_fama3.index,format='%Y%m%d')
updated_fama3/=100

updated_mom=pd.read_csv('updated_F-F_Momentum_Factor_daily.CSV')
updated_mom['Date']=updated_mom['Date'].apply(str)
updated_mom=updated_mom.set_index(['Date'])
updated_mom.index=pd.to_datetime(updated_mom.index,format='%Y%m%d')
updated_mom/=100

updated_fama=pd.concat([updated_fama3,updated_mom],axis=1)
updated_fama=pd.concat([fama_data,updated_fama])

# filter the FF returns to just the Stock return data
updated_fama=updated_fama.loc[rts.index][["Mkt-RF","SMB","HML",'Mom   ']]

In [22]:
# Use the optimal portforlio weights as initial weights
last_weight=SR_Portfolio['Weight']

# Weights
weights=[]
#  Factor weights
factorWeights=[]
# Portfolio Return
portfolio_rts=[]
# Beta of each stock
Beta=beta4
# Residual Return
residReturn=[]

for i in range(rts.shape[0]):
    # Calculate/Update and store Factor weights 
    factorWeights.append(last_weight@beta4.T)
    # store weights 
    weights.append(last_weight)

    # Update Weights by return
    last_weight=last_weight*(rts.iloc[i,1:]+1)
    # Calculate the portforlio return
    p_rt=last_weight.sum()
    # Normalize the wieghts back so sum = 1
    last_weight=last_weight/p_rt

    # Store the portforlio return
    portfolio_rts.append(p_rt-1)
    
    # Calculate and store the Residual return
    residReturn.append(portfolio_rts[-1]-factorWeights[-1]@updated_fama.iloc[i,:])
    
weights=pd.DataFrame(weights,index=rts.index)
factorWeights=pd.DataFrame(factorWeights,index=rts.index)
portfolio_rts=pd.DataFrame(portfolio_rts,index=rts.index,columns=['Portfolio'])
residReturn=pd.DataFrame(residReturn,index=rts.index,columns=['Alpha'])
# Calculate the total return
total_rt=(portfolio_rts+1).prod()-1

In [23]:
# Calculate the Carino K
k=np.log(total_rt+1)/total_rt
# Carino k_t is the ratio scaled by 1/K 
carinoK = np.log(1+portfolio_rts)/k/portfolio_rts

# # Used for position return attribution
# # Transform carinoK to dataframe to be multiplied by weights and rts
# carinoK_df=pd.DataFrame([carinoK['Portfolio']] * weights.shape[1],index=weights.columns).T
# # Calculate the return attribution of underlying positions (has been adjusted by carinoK_df)
# return_attribution=(weights * rts * carinoK_df).dropna(axis=1)

# Used for factor Attribution
# Calculate the return attribution of each Factor (has been adjusted by carinoK_df)
return_attribution_factors=(factorWeights * updated_fama).apply(carinoK['Portfolio'].multiply)
# Residual return attribution
return_attribution_factors['Alpha']=residReturn.apply(carinoK['Portfolio'].multiply)

In [24]:
# Calculate the total return attribution
total_rt_attribution_factors=return_attribution_factors.sum()

# Calculate the total return of each stock and portfolio
updated_fama['Alpha']=residReturn
total_rts_factors=((updated_fama+1).prod()-1)

# Combine the total_rt_attribution and total_rts together to compare with each other
attribution_df_factor=pd.concat([total_rt_attribution_factors,total_rts_factors],axis=1).T
attribution_df_factor.index=['TotalReturnAttribution','RealizedTotalReturn']

# Calulate the total sum of factor returns
attribution_df_factor.loc['TotalReturnAttribution','Portfolio']=attribution_df_factor.loc['TotalReturnAttribution'].sum()
attribution_df_factor.loc['RealizedTotalReturn','Portfolio']=total_rt[0]

# Validate whether the sum of Total Return Attribution is equal to the total Portfolio Return or not
# Check that the attribution sums back to the total Portfolio return
print(np.isclose(attribution_df_factor.loc['TotalReturnAttribution','Portfolio'],attribution_df_factor.loc['RealizedTotalReturn','Portfolio']))
attribution_df_factor

True


,Mkt-RF,SMB,HML,Mom,Alpha,Portfolio
TotalReturnAttribution,-0.044562,0.000103,0.002425,-0.000610,0.017580,-0.025063
RealizedTotalReturn,-0.058005,-0.001108,0.020274,0.008215,0.018044,-0.025063


# Realized Volatility Attribution

In [25]:
# Y is factor returns scaled by their weight at each time + alpha
Y = (factorWeights * updated_fama).dropna(axis=1)
Y = pd.concat([Y,residReturn],axis=1)

# Set up X with the Portfolio Return
X = pd.DataFrame(portfolio_rts)
X['Intersect']=1

# Calculate the Beta and discard the intercept
B = (linalg.inv(X.T@X)@X.T@Y).iloc[:-1,:]

# Component SD is Beta times the standard Deviation of the portfolio
cSD = B * portfolio_rts['Portfolio'].std()

#Check that the sum of component SD is equal to the portfolio SD
print(np.isclose(cSD.sum(axis=1),portfolio_rts['Portfolio'].std()))

# Add the portfolio SD to the cSD
cSD['Portfolio']=portfolio_rts['Portfolio'].std()
cSD.index=['VolatilityAttribution']

# attribution_df_factor.columns=columns
attribution_df_factor=pd.concat([attribution_df_factor,cSD])
attribution_df_factor



[ True]


,Mkt-RF,SMB,HML,Mom,Alpha,Portfolio
TotalReturnAttribution,-0.044562,0.000103,0.002425,-0.000610,0.017580,-0.025063
RealizedTotalReturn,-0.058005,-0.001108,0.020274,0.008215,0.018044,-0.025063
VolatilityAttribution,0.009948,-0.000111,-0.000728,0.000129,0.002687,0.011926


# Problem 3:
### Using the same data as Problem 1 and assuming a 0 mean return, fit a t distribution to each stock return series. Simulate the system using a Gaussian Copula. Find the Risk Parity portfolio using ES as the risk measure.

In [26]:
# Clear the memory
%reset -f

import pandas as pd
import numpy as np
from scipy.stats import norm,spearmanr,multivariate_normal,t
import datetime
import math
import itertools
from scipy import optimize
import sys
sys.path.append("..")
from RiskPackage.CalculateReturn import return_calculate
from scipy import linalg
from RiskPackage.ModelFitter import ModelFitter,T_mean0
from RiskPackage.RiskMetrics import RiskMetrics

## Calculate the optimal Sharpe ratio portfolio
### Using 4-factor Model to calculate the expected return to each stock

In [27]:
# parse the data of fama 3-factor
fama_data=pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
fama_data['Date']=fama_data['Date'].apply(str)
fama_data=fama_data.set_index(['Date'])
fama_data.index=pd.to_datetime(fama_data.index,format='%Y%m%d')
fama_data/=100
# parse the data of momentum
mom_data=pd.read_csv('F-F_Momentum_Factor_daily.CSV')
mom_data['Date']=mom_data['Date'].apply(str)
mom_data=mom_data.set_index(['Date'])
mom_data.index=pd.to_datetime(mom_data.index,format='%Y%m%d')
mom_data/=100
# combine them together
fama_data=pd.concat([fama_data,mom_data],axis=1)

# parse the return data 
rt=pd.read_csv('Dailyreturn.csv',index_col='Date')
rt.index=pd.to_datetime(rt.index)

In [28]:
stock_list=['AAPL','MSFT','BRK-B','CSCO','JNJ']

In [29]:
# find the intersected date between fama data and return data
intersected_date=list(set(rt.index).intersection(set(fama_data.index )))
excess_r=rt.sub(fama_data.loc[intersected_date,'RF'].sort_index(axis=0),axis=0)
# get the return and excess return data of given stock list
rt=rt[stock_list]
excess_r=excess_r[stock_list]

In [30]:
# Ge the X to do the regression (4 facotrs)
x4=pd.concat([fama_data.loc[intersected_date,:'HML'],fama_data.loc[intersected_date,'Mom   ']],axis=1)
x4['intersect']=1
x4=x4.sort_index(axis=0)
# Calculate the beta
X4=np.array(x4)
beta4=linalg.inv(X4.T@X4)@X4.T@excess_r
beta4.index=x4.columns

In [31]:
# past 10 years of factor returns
expected_fama_factor=fama_data.loc[rt.index[-1]-pd.DateOffset(years=10):rt.index[-1],:].mean()
expected_fama_factor['intersect']=1

# past 10 years of risk-free rate
expected_RF=expected_fama_factor['RF']

# 4-factor returns
# expected_fama_factor4 = expected_fama_factor[["Mkt-RF","SMB","HML",'Mom   ',"intersect"]]
beta4=beta4.iloc[:4,:]
expected_fama_factor4 = expected_fama_factor[["Mkt-RF","SMB","HML",'Mom   ']]

# Expected return of each stock
expected_return4=expected_fama_factor4 @ beta4
expected_return4=np.log(1+expected_return4)*255
# Covariance
cov=np.log(1+excess_r).cov()*255

## Super Efficient, Maximum Sharpe Ratio, portfolio

In [32]:
def super_efficient_portfolio(expected_rts,cov):
    '''Given a target return, use assets to find the optimal portfolio with lowest risk'''
    rf=0.0025
    fun=lambda wts: -(wts@expected_rts-rf)/np.sqrt(wts@cov@wts)
    x0 = np.full(expected_rts.shape[0],1/expected_rts.shape[0])
    cons = [{'type':'ineq', 'fun':lambda x:x},
        {'type':'eq', 'fun':lambda x:sum(x)-1}]
    bounds = [(0, 1) for _ in range(expected_rts.shape[0])]
    res = optimize.minimize(fun, x0, method='SLSQP',bounds=bounds,constraints=cons)
    return res

In [33]:
# optimal portfolio
res=super_efficient_portfolio(expected_return4,cov)
SR_Portfolio=pd.DataFrame(index=stock_list)
SR_Portfolio['Weight']=res.x
SR_Portfolio['return_attribution']=res.x*expected_return4

## Volatility Risk Parity Portfolio 

In [34]:
def riskBudget(w,cov):
    '''Calculate the portion of risk each stock of portfolio has. The sum of result is 1'''
    portfolioStd=np.sqrt(w@cov@w)
    Csd=w*(cov@w)/portfolioStd
    return Csd/portfolioStd

In [35]:
SR_Portfolio['RiskPortion']=riskBudget(SR_Portfolio['Weight'],cov)
SR_Portfolio

,Weight,return_attribution,RiskPortion
AAPL,0.100792,0.014285,0.118004
MSFT,0.209487,0.035649,0.295352
BRK-B,0.438283,0.048322,0.397046
CSCO,0.081217,0.011085,0.091508
JNJ,0.170222,0.012090,0.098090


In [36]:
def RiskParity(cov):
    '''Given a target return, use assets to find the optimal portfolio with lowest risk'''
    fun=lambda w: (w*(cov@w)/np.sqrt(w@cov@w)).std()
    x0 = np.full(cov.shape[0],1/cov.shape[0])
    cons = [{'type':'ineq', 'fun':lambda x:x},
        {'type':'eq', 'fun':lambda x:sum(x)-1}]
    bounds = [(0, 1) for _ in range(cov.shape[0])]
    res = optimize.minimize(fun, x0, method='SLSQP',bounds=bounds,constraints=cons)
    return res

In [37]:
SR_Portfolio['return_attribution']=res.x*expected_return4

In [38]:
# Function to calculate the Csd given the weight
CsdFun=lambda w:w*(cov@w)/np.sqrt(w@cov@w)

# Risk Parity portfolio
res=RiskParity(cov)
RP_Portfolio=pd.DataFrame(index=stock_list)
RP_Portfolio['Weight']=res.x
RP_Portfolio['CER']=res.x * expected_return4 # component expected return 
RP_Portfolio['CSD']=CsdFun(res.x) # component stanard deviation
RP_Portfolio['RiskBudget']=riskBudget(RP_Portfolio['Weight'],cov) 
RP_Portfolio['AssetVol']=np.sqrt(np.diag(cov))
RP_Portfolio

,Weight,CER,CSD,RiskBudget,AssetVol
AAPL,0.154198,0.021854,0.022686,0.199988,0.255815
MSFT,0.142554,0.024259,0.022685,0.199977,0.255414
BRK-B,0.265171,0.029236,0.022690,0.200020,0.151584
CSCO,0.150787,0.020581,0.022688,0.200002,0.233510
JNJ,0.287290,0.020405,0.022689,0.200013,0.149137


## ES Risk Parity Portfolio

In [39]:
# Remove the mean of excess return of each stock
Y = excess_r-excess_r.mean()

# Use the CDF to transform the data to uniform universe
U=[]
Model_T=[]
for i in range(Y.shape[1]):
    params=t.fit(Y.iloc[:,i].values)
    Model_T.append(t(df=params[0],loc=params[1],scale=params[2]))
    U.append(Model_T[i].cdf(Y.iloc[:,i]))

nSim = 10000

# Gaussian Copula -- Technically we should do 255 days ahead...
# Use the standard normal quantile function to transform the uniform to normal 
Z=norm.ppf(U)
Z=pd.DataFrame(Z,index=Y.columns).T
# Spearman correlation
corr_spearman = spearmanr(Z,axis=0)[0]
# Simulate Normal & Transform to uniform
simU=norm.cdf(multivariate_normal.rvs(cov=corr_spearman, size=nSim))
simU=pd.DataFrame(simU,columns=Y.columns)
# Transform to T Distribution
simReturns=[]
for i in range(Y.shape[1]):
    simReturns.append(Model_T[i].ppf(simU.iloc[:,i]))
simReturns=pd.DataFrame(simReturns,index=Y.columns).T


In [40]:
def CES(weights,simReturns):
    '''Function for the component ES'''
    ES=lambda w: RiskMetrics.ES_historical(np.array(simReturns @ w))
    ini_ES=ES(weights)
    e=1e-6
    n=len(weights)
    weightsMat=np.array([weights]*n)+np.diag([e]*n)
    # component ES
    Ces=[]
    for i in range(n):
        Ces.append(weights[i]*(ES(weightsMat[i])-ini_ES)/e)
    return Ces

def SSE_CES(weights,simReturns):
    Ces=CES(weights,simReturns)
    # std(SSE) of CES 
    return pd.DataFrame(Ces).std()[0]

x0 = np.full(simReturns.shape[1],1/simReturns.shape[1])
cons = [{'type':'ineq', 'fun':lambda x:x},
    {'type':'eq', 'fun':lambda x:sum(x)-1}]
bounds = [(0, 1) for _ in range(cov.shape[0])]
res = optimize.minimize(SSE_CES, x0, method='SLSQP',bounds=bounds,constraints=cons,args=(simReturns))

In [43]:
ES=lambda w: RiskMetrics.ES_historical(np.array(simReturns @ w))
# Risk Parity portfolio
ES_RP_Portfolio=pd.DataFrame(index=stock_list)
ES_RP_Portfolio['Weight']=res.x
ES_RP_Portfolio['CER']=res.x * expected_return4 # component expected return 
ES_RP_Portfolio['CSD']=CsdFun(res.x) # component stanard deviation
ES_RP_Portfolio['RiskBudget']=riskBudget(ES_RP_Portfolio['Weight'],cov) 
ES_RP_Portfolio['AssetVol']=np.sqrt(np.diag(cov))
ES_RP_Portfolio['CES']=CES(res.x,simReturns)
print("------ ES Risk Parity Portfolio -------")
ES_RP_Portfolio

------ ES Risk Parity Portfolio -------


,Weight,CER,CSD,RiskBudget,AssetVol,CES
AAPL,0.142281,0.020165,0.019733,0.175846,0.255815,0.003038
MSFT,0.134850,0.022948,0.020508,0.182751,0.255414,0.003037
BRK-B,0.268402,0.029592,0.023773,0.211847,0.151584,0.003025
CSCO,0.151790,0.020718,0.022886,0.203938,0.233510,0.003028
JNJ,0.302677,0.021498,0.025318,0.225618,0.149137,0.003028


In [44]:
print("------ Vol Risk Parity Portfolio -------")
RP_Portfolio

------ Vol Risk Parity Portfolio -------


,Weight,CER,CSD,RiskBudget,AssetVol
AAPL,0.154198,0.021854,0.022686,0.199988,0.255815
MSFT,0.142554,0.024259,0.022685,0.199977,0.255414
BRK-B,0.265171,0.029236,0.022690,0.200020,0.151584
CSCO,0.150787,0.020581,0.022688,0.200002,0.233510
JNJ,0.287290,0.020405,0.022689,0.200013,0.149137


### AAPL returns, with a large DF, are effectively normally distributed in this simulation. Same with BRK-B. 
### The 2 stocks with the lowest fitted df are both reduced in weight as their fat tails contribute more to tail risk.